In [60]:
from itertools import chain, combinations
from copy import deepcopy

In [157]:
class Log:
    def __init__(self, logfile):
        self.logfile = logfile
        self.traces = self._parse_logfile()
        
        self.t_all = set()
        self.t_i = set()
        self.t_o = set()
        
        # footprint relations
        self.df = set() # directly follows
        self.cd = set() # causally dependent
        self.ncd = set() # not causally dependent
        self.pl = set() # parallel
        
        self.x_l = set()
        self.y_l = set()
    
    def _parse_logfile(self):
        # TODO
        l = [["a","b","c","d"],["a","c","b","d"],["a","e","d"]]
        return l

#### 1-3) Activity set extraction and footprint matrix generation

In [181]:
class Miner:
    def __init__(self, algorithm="standard"):
        self.algorithm = algorithm
        self.log = None
        
    def run_miner(self,log):
        self.log = log
        
        # run algorithm
        self.extract_activity_sets()
        self.generate_footprint()
        self.create_x_l()
        self.create_y_l()
    
    def extract_activity_sets(self):
        for trace in self.log.traces:
            for i, item in enumerate(trace):
                self.log.t_all.add(item)
                if i == 0:
                    self.log.t_i.add(item)
                elif i == len(trace)-1:
                    self.log.t_o.add(item)
    
    def generate_footprint(self):
        # directly follows
        self.log.df = self._directly_follows(self.log.traces)
        
        for a1 in self.log.t_all:
            for a2 in self.log.t_all:
                if (a1,a2) not in self.log.cd and (a1,a2) not in self.log.ncd and (a1,a2) not in self.log.pl:
                    if (a1,a2) in self.log.df:
                        if (a2,a1) in self.log.df:
                            # parallel: a1 -> a2 & a2 -> a1
                            self.log.pl.add((a1,a2))
                        else:
                            # dependent: a1 -> a2 & !(a2 -> a1)
                            self.log.cd.add((a1,a2))
                    else:
                        if (a2,a1) not in self.log.df:
                            # independent: !(a1 -> a2) & !(a2 -> a1)
                            self.log.ncd.add((a1,a2))
                            
        
    def _directly_follows(self,traces):
        df = set()
        for t in traces:
            for i, a in enumerate(t):
                if i != len(t)-1:
                    df.add((a,t[i+1]))
        return df

    
    def create_x_l(self):
        # TODO: make it more efficient (don't do exhaustive search)
        x_l = set()
        subsets = subset_generation(self.log.t_all)
        for ss1 in subsets:
            for ss2 in subsets:
                if self._check_set_independence(ss1,self.log.ncd):
                    a = ss1
                    if self._check_set_independence(ss2,self.log.ncd):
                        b = ss2
                        if self._check_set_relation(a,b,self.log.cd):
                            x_l.add((a,b))
        self.log.x_l = x_l
    
    def _check_set_independence(self,s,ncd):
        for a1 in s:
            for a2 in s:
                if (a1,a2) not in ncd:
                    return False
        return True
    
    def _check_set_relation(self,s1,s2,cd):
        for a1 in s1:
            for a2 in s2:
                if (a1,a2) not in cd:
                    return False
        return True
    
    def create_y_l(self):
        self.log.y_l = remove_non_maximum_sets(self.log.x_l)

#### 4) Place Construction -> X_L
Construct places p(A,B) such that A is set of in input transitions and B is set of output transitions.  
* elements of A should not follow eachother (not causally dependent)
* elements of B should not follow eachother (not causally dependent)
* any of the elements in A can be followed by any of the elements in B

In [173]:
# generate all possible subsets of activities
# (depending on log size it can be expensive)

def subset_generation(activity_set):
    return list(chain.from_iterable(combinations(activity_set,r) for r in range(1,len(activity_set)+1)))

#### 5) Removal of Non-Maximum Pairs -> Y_L

In [174]:
def remove_non_maximum_sets(sets):
    maximal_pairs = deepcopy(sets)
    for s1 in sets:
        for s2 in sets:
            if set(s1[0]).issubset(set(s2[0])):
                if set(s1[1]).issubset(set(s2[1])):
                    if s1[0] != s2[0]:
                        maximal_pairs.discard(s1)
                    elif s1[1] != s2[1]:
                        maximal_pairs.discard(s1)
    return maximal_pairs

#### 6-7) Source & Sink & Transitions
* add unique source place which all start transitions in T_l get as input
* add unique sink place which all end transitions in T_o get as output
* all pairs become places with A as input transitions and B as output transitions

In [175]:
class Place:
    def __init__(self,in_set,out_set):
        self.in_set = in_set
        self.out_set = out_set
        
    def __repr__(self):
        return f"{self.in_set} -- {self.out_set}"

In [178]:
# Example
lg = Log("dummy.txt")
miner = Miner()
miner.run_miner(lg)   # in place

In [179]:
# create P_l transitions

p_l = []
# add source
p_l.append((Place(None,lg.t_i)))
# add transitions
p_l.extend([Place(p[0],p[1]) for p in lg.y_l])
# add sink
p_l.append((Place(lg.t_o, None)))

In [180]:
for t in p_l:
    print(t)

None -- {'a'}
('a',) -- ('e', 'c')
('e', 'b') -- ('d',)
('e', 'c') -- ('d',)
('a',) -- ('e', 'b')
{'d'} -- None


# Questions / To Discuss
* Output Format / Visualisation
* class & function structure
* TODO: parse input file